In [ ]:
# Część 1: Utwórz model, który klasyfikuje obrazy na dwie klasy – pochmurno, wschód słońca

# Importowanie niezbędnych bibliotek
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score
import numpy as np
import seaborn as sns
import zipfile
import os
import requests
import shutil

# Definicje funkcji pomocniczych

# Funkcja do pobierania i rozpakowywania zbioru danych
def download_and_extract_data(url, extract_to='.'):
    local_zip = os.path.join(extract_to, 'weather.zip')
    if not os.path.exists(local_zip):
        response = requests.get(url, stream=True)
        with open(local_zip, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        with zipfile.ZipFile(local_zip, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
    else:
        print("Dataset already downloaded.")

# Pobieranie i rozpakowywanie zbioru danych
download_and_extract_data("http://www.kasprowski.pl/datasets/weather.zip")

# Ścieżki do folderów z danymi treningowymi i testowymi
base_dir = './weather'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Ustawienia ImageDataGenerator dla dwóch klas (pochmurno i wschód słońca)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training',
    classes=['cloudy', 'sunrise']
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    classes=['cloudy', 'sunrise']
)

# Budowanie modelu konwolucyjnej sieci neuronowej
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Ocena modelu
validation_generator.reset()
preds = model.predict(validation_generator)
preds = (preds > 0.5).astype(int)
y_true = validation_generator.classes
y_pred = preds

# Wyświetlanie wyników
print("Accuracy: ", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred))
print("Cohen Kappa Score: ", cohen_kappa_score(y_true, y_pred))

# Wizualizacja macierzy zamieszania
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Część 2: Utwórz model, który klasyfikuje obrazy na wszystkie cztery klasy

# Ustawienia ImageDataGenerator dla wszystkich czterech klas
datagen_all = ImageDataGenerator(rescale=1./255, validation_split=0.25)

train_generator_all = datagen_all.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_all = datagen_all.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Budowanie modelu konwolucyjnej sieci neuronowej dla czterech klas
model_all = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

# Kompilacja modelu
model_all.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history_all = model_all.fit(train_generator_all, epochs=10, validation_data=validation_generator_all)

# Ocena modelu
validation_generator_all.reset()
preds_all = model_all.predict(validation_generator_all)
y_true_all = validation_generator_all.classes
y_pred_all = np.argmax(preds_all, axis=1)

# Wyświetlanie wyników
print("Accuracy: ", accuracy_score(y_true_all, y_pred_all))
print("Confusion Matrix:\n", confusion_matrix(y_true_all, y_pred_all))
print("Classification Report:\n", classification_report(y_true_all, y_pred_all))
print("Cohen Kappa Score: ", cohen_kappa_score(y_true_all, y_pred_all))

# Wizualizacja macierzy zamieszania
sns.heatmap(confusion_matrix(y_true_all, y_pred_all), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
